# Import packages

In [1]:
import MetaTrader5 as mt
import pandas as pd
import ml_collections
import yaml

from datetime import datetime
import pytz
import sys

sys.path.insert(0, 'D:/Intraday_trading')
from src.support import *

import time
from IPython.display import clear_output

In [2]:
config_dir = 'D:/Intraday_trading/configs/config.yaml'
with open(config_dir) as file:
    config = ml_collections.ConfigDict(yaml.safe_load(file))


In [3]:
acc_config = config.real_acc

# Login account

In [4]:
login_metatrader(acc_config = acc_config)

# Deals monitor 

In [5]:
from_date=datetime(2024,1,1)
to_date=datetime.now()

deals = mt.history_deals_get(from_date, to_date, group="XAUUSDm")

df_deals = pd.DataFrame(deals, columns = deals[0]._asdict().keys())

df_deals['time'] = pd.to_datetime(df_deals['time'], unit = 's')
df_deals['time_msc'] = pd.to_datetime(df_deals['time_msc'], unit = 'ms')

df_deals['VN_time'] = df_deals['time'] + pd.Timedelta('7 hours')
df_deals['VN_time_msc'] = df_deals['time_msc'] + pd.Timedelta('7 hours')


In [6]:
ref_deals_entry = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_entry.csv', encoding_errors = 'ignore')
ref_deals_reason = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_reason.csv', encoding_errors = 'ignore')
ref_deals_type = pd.read_csv('D:/Intraday_trading/src/ref_tables/ref_deals_type.csv', encoding_errors = 'ignore')


In [7]:
df_deals = df_deals.merge(ref_deals_entry, how = 'left', left_on = 'type', right_on = ref_deals_entry.index)
df_deals = df_deals.merge(ref_deals_reason, how = 'left', left_on = 'type', right_on = ref_deals_reason.index)
df_deals = df_deals.merge(ref_deals_type, how = 'left', left_on = 'type', right_on = ref_deals_type.index)

In [8]:
df_deals.tail(6)

,ticket,order,time,time_msc,type,entry,magic,position_id,reason,volume,price,commission,swap,profit,fee,symbol,comment,external_id,VN_time,VN_time_msc,IDENTIFIER_ENTRY,DESCRIPTION_ENTRY,IDENTIFIER_REASON,DESCRIPTION_REASON,IDENTIFIER_TYPE,DESCRIPTION_TYPE
115,83636196,302562186,2024-06-28 07:33:08,2024-06-28 07:33:08.691,0,0,0,302562186,1,0.03,2325.098,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 14:33:08,2024-06-28 14:33:08.691,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
116,83663964,302683525,2024-06-28 08:38:29,2024-06-28 08:38:29.924,0,0,0,302683525,1,0.03,2331.499,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 15:38:29,2024-06-28 15:38:29.924,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
117,83718903,302801066,2024-06-28 11:02:45,2024-06-28 11:02:45.553,0,0,0,302801066,1,0.03,2337.034,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 18:02:45,2024-06-28 18:02:45.553,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
118,83720770,302808511,2024-06-28 11:06:36,2024-06-28 11:06:36.043,0,0,0,302808511,1,0.03,2336.663,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 18:06:36,2024-06-28 18:06:36.043,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy
119,83855207,303120409,2024-06-28 14:43:15,2024-06-28 14:43:15.855,1,1,0,302801066,4,0.03,2322.824,0.0,0.0,-42.63,0.0,XAUUSDm,[sl 2322.824],,2024-06-28 21:43:15,2024-06-28 21:43:15.855,DEAL_ENTRY_OUT,Entry out,DEAL_REASON_MOBILE,The deal was executed as a result of activatio...,DEAL_TYPE_SELL,Sell
120,83891728,303202600,2024-06-28 16:05:55,2024-06-28 16:05:55.011,0,0,0,303202600,1,0.03,2330.628,0.0,0.0,0.00,0.0,XAUUSDm,,,2024-06-28 23:05:55,2024-06-28 23:05:55.011,DEAL_ENTRY_IN,Entry in,DEAL_REASON_CLIENT,The deal was executed as a result of activatio...,DEAL_TYPE_BUY,Buy


# Orders monitor

In [9]:
from_date=datetime(2024,6,15)
to_date=datetime.now()
# get deals for symbols whose names contain "XAUUSDm" within a specified interval
orders = mt.history_orders_get(from_date, to_date, group="*XAUUSDm*")

df_orders = pd.DataFrame(orders, columns = orders[0]._asdict().keys())
df_orders['time_setup'] = pd.to_datetime(df_orders['time_setup'], unit = 's')
df_orders['VN_time'] = df_orders['time_setup'] + pd.Timedelta('7 hours')

In [11]:
df_orders.tail(6)

,ticket,time_setup,time_setup_msc,time_done,time_done_msc,time_expiration,type,type_time,type_filling,state,magic,position_id,position_by_id,reason,volume_initial,volume_current,price_open,sl,tp,price_current,price_stoplimit,symbol,comment,external_id,VN_time
38,302683525,2024-06-28 08:38:29,1719563909923,1719563909,1719563909924,0,0,0,1,4,0,302683525,0,1,0.03,0.00,0.000,0.000,0.000,2331.499,0.0,XAUUSDm,,,2024-06-28 15:38:29
39,302799243,2024-06-28 10:56:52,1719572212442,1719572788,1719572788395,0,2,0,2,2,0,0,0,1,0.03,0.03,2335.792,2324.339,2343.678,2336.521,0.0,XAUUSDm,,,2024-06-28 17:56:52
40,302801066,2024-06-28 10:59:10,1719572350390,1719572565,1719572565553,0,4,0,2,4,0,302801066,0,1,0.03,0.00,2337.034,2325.261,2343.684,2337.088,0.0,XAUUSDm,,,2024-06-28 17:59:10
41,302808511,2024-06-28 11:06:36,1719572796041,1719572796,1719572796043,0,0,0,1,4,0,302808511,0,1,0.03,0.00,0.000,0.000,0.000,2336.663,0.0,XAUUSDm,,,2024-06-28 18:06:36
42,303120409,2024-06-28 14:43:15,1719585795853,1719585795,1719585795855,0,1,0,1,4,0,302801066,0,4,0.03,0.00,2322.824,0.000,0.000,2322.824,0.0,XAUUSDm,[sl 2322.824],,2024-06-28 21:43:15
43,303202600,2024-06-28 15:56:06,1719590166435,1719590755,1719590755011,0,4,0,2,4,0,303202600,0,1,0.03,0.00,2330.628,2320.555,2343.507,2330.690,0.0,XAUUSDm,,,2024-06-28 22:56:06


# Data

## OHLCV interval data

In [ ]:
# Compute now date
from_date = datetime(2001, 1, 1)
to_date = datetime(2024, 4, 11)


rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M5, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_M15, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_H1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])



rates = mt.copy_rates_range("EURUSDm", mt.TIMEFRAME_D1, from_date, to_date)
df_rates = pd.DataFrame(rates)

df_rates["time"] = pd.to_datetime(df_rates["time"], unit="s")
print(df_rates['time'][0])

## Ticks data

In [ ]:
# https://thiagomarzagao.com/2021/05/12/mt5/

In [ ]:
# Compute now date
from_date = datetime(2020, 4, 10)
to_date = datetime(2024, 4, 11)

# Extract n Ticks before now
ticks = mt.copy_ticks_range("EURUSDm", from_date, to_date, mt.COPY_TICKS_ALL)

# Transform Tuple into a DataFrame
df_ticks = pd.DataFrame(ticks)

# Convert number format of the date into date format
df_ticks["time"] = pd.to_datetime(df_ticks["time"], unit="s")


df_ticks